# 生成20个氨基酸OneHot图像

In [ ]:
import matplotlib.pyplot as plt
import scipy.io as sio
from PIL import Image
data=sio.loadmat('data\\PDNA-224-ONEHOT-11-P.mat')
d = data['data']
N = d.shape[0]
for i in range(N):
    img = Image.fromarray(d[i]*255,'L')
    img.save('data\\img\\positive\\' + str(i) + '.jpg','jpeg')

In [ ]:
import matplotlib.pyplot as plt
import scipy.io as sio
from PIL import Image
data=sio.loadmat('data\\PDNA-224-ONEHOT-11-N.mat')
d = data['data']
N = d.shape[0]
for i in range(N):
    img = Image.fromarray(d[i]*255,'L')
    img.save('data\\img\\negro\\' + str(i) + '.jpg','jpeg')

In [3]:
import matplotlib.pyplot as plt
import scipy.io as sio
from PIL import Image
data = sio.loadmat('data\\PDNA-224-OneHot-15.mat')
d = data['data']
Y = data['target']
N = d.shape[0]
n = 0
m = 0
for i in range(N):
    img = Image.fromarray(d[i]*255,'L')
    if Y[i][0]==1:
        img.save('img-15\\negative\\' + str(n) + '.jpg', 'jpeg')
        n = n + 1
    else:
        img.save('img-15\\positive\\' + str(m) + '.jpg', 'jpeg')
        m = m + 1

# 生成两联体OneHot图像

In [10]:
import os
import numpy as np
from Bio import SeqIO
import matplotlib.pyplot as plt

In [1]:
alphabet = 'ACDEFGHIKLMNPQRSTVWYX'
daa = [(x+y) for x in alphabet for y in alphabet]

#把一个长度为L的氨基酸序列转换为一个矩阵型数组。按照20个氨基酸的两联体以ONE-HOT方式编码
#矩阵的列数是441
#矩阵的行数根据参数r，有：(L-1)+(L-2)+...+(L-r)=rL-(r+1)*r/2
def seq2DaaOneHotArray(sequence, r):
    L = len(sequence)
    N = r*L - ((r+1)*r)/2
    result = np.zeros(shape=(int(N), 441))
    m = 0
    for i in range(r):
        for j in range(L-i-1):
            aa = sequence[j]+sequence[j+i+1]
            k = daa.index(aa)
            result[m][k] = 1
            m = m + 1  
    return result

In [4]:
#读入序列文件和位点文件
from Bio import SeqIO
def loadBindingsites(fastaFile, siteFile):
    #读序列文件，每一个序列构成字典的一项，
    #key：序列的id
    #value: 氨基酸序列的字母字符串
    data = {}
    for seq_record in SeqIO.parse(fastaFile, 'fasta'):
        data[seq_record.id] = seq_record.seq
    
    #读位点文件
    bindingsites = {}
    with open(siteFile, 'r') as pbsreader:
        i = 0
        for line in pbsreader:
            i = i + 1
            line = line.strip()
            if '>' in line:
                sid = line[1:]
            else:
                sites = line.split()
                bs = []
                for site in sites:
                    bs.append( int(site))
            if i%2 == 0:
                bindingsites[sid] = bs
      
    return (data, bindingsites)

In [5]:
data,bindingsites = loadBindingsites('../data/PDNA-224.fasta','../data/PDNA-224-binding-sites.txt')

In [21]:
# 构建序列样本集图像
import matplotlib.pyplot as plt
from PIL import Image

def splitDatasets(data, bindingsites,ws,r):
    i1, i2 = 0,0
    for key in data:
        sites = bindingsites[key]
        p = data[key]
        seqlen = len(p)
        for j in range(seqlen):
            if j < ws:
                seq = str(p[j-ws:]) + str(p[0: ws+j+1])
            elif j > seqlen - ws -1:
                seq = str(p[j-ws:j]) + str(p[j:]) + str(p[0:ws-seqlen+j+1])
            else:
                seq = str(p[j-ws:j+ws+1])
            m = seq2DaaOneHotArray(seq, r)
            img = Image.fromarray(m*255,'L')
            if j in sites:
                img.save('../data/img-15-15/positive/' + str(i1) + '.jpg','jpeg')
                i1 += 1
            else:
                #negative.append(m)
                img.save('../data/img-15-15/negative/' + str(i2) + '.jpg','jpeg')
                i2 += 1

In [23]:
splitDatasets(data, bindingsites,15,15)